In [1]:
from pathlib import Path
import pandas as pd
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import seaborn as sns


In [2]:
folder_path = r"D:\Lorenzo\opto"
file_name = 'intensity_and_speed_df_'
strain_name = 'merge_reps'
strain_file_name =  file_name + strain_name + ".csv"
worm_data_df = pd.read_csv(os.path.join(folder_path, strain_file_name))
worm_data_df

,time,n_worm,intensity_gcamp,intensity_gcamp_uncorrected,intensity_mKate,centroid_x,centroid_y,replicate,distance,speed,normalised_intensity,strain
0,0,1,104.255639,110.227273,99.781955,11,117,1,NaN,NaN,1.044835,2523
1,1,1,102.214612,104.000000,99.968037,12,118,1,1.414214,0.072266,1.022473,2523
2,2,1,101.724382,104.000000,98.745583,12,118,1,0.000000,0.000000,1.030166,2523
3,3,1,103.957895,104.625000,101.136842,12,111,1,7.000000,0.357700,1.027893,2523
4,4,1,101.308176,107.000000,99.924528,11,118,1,7.071068,0.361332,1.013847,2523
...,...,...,...,...,...,...,...,...,...,...,...,...
7545,45,29,128.145454,130.571429,134.636364,37,68,3,106.103723,5.421900,0.951789,2764
7546,46,29,126.621212,124.600000,133.030303,36,96,3,28.017851,1.431712,0.951822,2764
7547,47,29,122.327869,123.881356,130.950820,80,30,3,79.322128,4.053361,0.934151,2764
7548,48,29,124.156250,133.255814,131.953125,148,63,3,75.584390,3.862362,0.940912,2764


In [3]:
worm_data_df['normalised_intensity'] = pd.to_numeric(worm_data_df['normalised_intensity'], errors='coerce')
worm_data_df['speed'] = pd.to_numeric(worm_data_df['speed'], errors='coerce')

sns.set_theme(style='darkgrid')

sns.lineplot(
    data=worm_data_df,
    x="time", y="speed", 
    hue="strain",
    style="condition",
    # markers=True,
    palette='colorblind'
    # dashes = False
    
)

ValueError: Could not interpret value `condition` for parameter `style`

In [7]:
# Step 1: Create a new column in the DataFrame
worm_data_df['condition_experiment'] = worm_data_df['condition'] + "_" + worm_data_df['experiment']

# Step 2: Use the new column for styling
sns.set_theme(style='darkgrid')

sns.lineplot(
    data=worm_data_df,
    x="time", y="normalised_intensity", 
    hue="condition_experiment",
    style="strain",  # Use the combined column
    # markers=True,
    palette='colorblind'
    # dashes = False
)

KeyError: 'experiment'

In [4]:
# Group by strain
# grouped_norm = worm_data_df.groupby(['condition', 'experiment','strain', 'time'])
grouped_norm = worm_data_df.groupby(['strain', 'time'])
# Calculate the mean normalized intensity for each group
mean_intensity_by_group = grouped_norm['normalised_intensity'].mean()
# Calculate the standard deviation for each group
std_intensity_by_group = grouped_norm['normalised_intensity'].std()
# Count the number of worms for each group
count_intensity_by_group = grouped_norm['normalised_intensity'].count()
# Calculate the standard error for each group
standard_error_intensity_by_group = std_intensity_by_group / np.sqrt(count_intensity_by_group)

# Calculate the mean normalized intensity for each group
mean_speed_by_group = grouped_norm['speed'].mean()
# Calculate the standard deviation for each group
std_speed_by_group = grouped_norm['speed'].std()
# Count the number of worms for each group
count_speed_by_group = grouped_norm['speed'].count()
# Calculate the standard error for each group
standard_error_speed_by_group = std_speed_by_group / np.sqrt(count_speed_by_group)

# Create a new DataFrame with the strain, day, mean, and standard error
avg_norm_df = pd.DataFrame({
    'time': mean_intensity_by_group.index.get_level_values('time'),
    'strain': mean_intensity_by_group.index.get_level_values('strain'),
    #'condition':mean_intensity_by_group.index.get_level_values('condition'),
    #'experiment':mean_intensity_by_group.index.get_level_values('experiment'),
    'mean_normalized_intensity': mean_intensity_by_group.values,
    'standard_error_intensity': standard_error_intensity_by_group.values,
    'mean_speed': mean_speed_by_group.values,
    'standard_error_speed': standard_error_speed_by_group.values
})

avg_norm_df.to_csv(Path(folder_path).joinpath('intensity_avg_error.csv'), index=False) #False exclude index column
avg_norm_df

,time,strain,mean_normalized_intensity,standard_error_intensity,mean_speed,standard_error_speed
0,0,2523,0.948216,0.007690,2.714545,0.192329
1,1,2523,0.947804,0.007441,1.842707,0.204688
2,2,2523,0.952710,0.007872,1.967032,0.226436
3,3,2523,0.948773,0.007120,1.711350,0.191093
4,4,2523,0.949857,0.007220,1.704622,0.211748
...,...,...,...,...,...,...
95,45,2764,1.051558,0.007638,1.582688,0.179817
96,46,2764,1.052651,0.007514,1.634599,0.174875
97,47,2764,1.050669,0.007374,1.454431,0.142330
98,48,2764,1.046418,0.007818,1.458236,0.164913


In [5]:
grouped_norm